<a href="https://colab.research.google.com/github/Carapet07/Data-Science-projects/blob/main/First_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##My first NLP project

In [2]:
import os, pathlib, shutil, random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  2128k      0  0:00:38  0:00:38 --:--:-- 4375k
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [4]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

# Move 20% of the training data into new validation directory
for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files) # shuffle files for randomness
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]

  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [5]:
batch_size = 32

# Create datasets from directories, automatically batching text samples
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [6]:
for text_batch, label_batch in train_ds.take(1):  # Take one batch
    for i in range(2):  # Show the first 5 examples
        print(f"Text: {text_batch.numpy()[i].decode('utf-8')}")
        print(f"Label: {label_batch.numpy()[i]}")
        print("=" * 50)


Text: Nahhh! Leila (Grace Mills) is a teenager turned on to Satan (and LSD) by her archaeologist fiancé Richard. There's a neighborhood hippie demon cult hanging out at the local decrepit ancient castle, where Leila and Richard drink blood, drop drugs, join in sex orgies, dance to lame psychedelic rock and participate in black mass ceremonies where the guys wear pants, masks and capes and the women don't wear anything at all. Unfortunately, these kind of extracurricular activities have left Leila open to demonic possession from the dreaded "spirit of evil." Leila also comes from a screwed up family, which doesn't help either. Her older brother John is a recluse who seems to be in love with her. Her mother Patricia (Maria Perschy) is depressed because she thinks she was responsible for the father's death. To top it off, her sister (Maria Kosti) is a slutty semi pro golfer named (gasp!) Debbie Gibson. There's plenty of hired help around also to waste more time. Two maids (a young one who

In [7]:
max_length = 600 # max length of a movie review
max_tokens = 20000 # max number of unique words to consider

# Create a TextVectorization layer that converts text into integer sequences
# eg: "dog": [0.2, 0.5, -0.7, 0.21... embedding_dim]
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_length
)

# Extract only the text fron train_ds, removing lables (0, 1)
text_only_train_ds = train_ds.map(lambda x, y: x)

# This line adapts the TextVectorization layer to the dataset, learning the vocabulary
# by assigning each word in the dataset a unique integer index
text_vectorization.adapt(text_only_train_ds)

# apply it to every dataset we have
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)

In [7]:
for inputs, targets in train_ds:
  print("input shape: ", inputs.shape)
  print("input dtype: ", inputs.dtype)
  print("targets shape: ", targets.shape)
  print("targets dtype: ", targets.dtype)
  print('inputs[0]: ', inputs[0])
  print('targets[0]: ', targets[0])
  break

input shape:  (32,)
input dtype:  <dtype: 'string'>
targets shape:  (32,)
targets dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor(b'My friend\'s mom used to work at a video store and got to preview movies before they came out, so when she brought home The Convent, a horror movie, i couldn\'t wait to watch it. Given that it\'s supposed to be scary but is actually downright hilarious, I can say that in some weird way, I like this movie. <br /><br />yes, the acting is bad, and yes, it\'s the cheapest movie i\'ve ever seen, but it\'s so damn funny! "WHAT, ARE YOU SMOKING CA-RACK?!" i didn\'t know this movie even was ever released... i figured it was too bad... <br /><br />Yeah, so... overall the movie is pretty bad (you gotta admit that much at least) but I promise you, you will get a good laugh out of it.<br /><br />*this movie kinda sucks but it\'s good for a laugh... especially that guy that holds the \'dagger of despair\'.. THE DAGGER OF DESPAAAAAAIR!', shape=(), dtype=string)
targets[0

In [8]:
# downlaod GloVe precomputed database for words ebedding task
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-02-20 18:53:02--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-02-20 18:53:02--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-02-20 18:53:03--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

Parse the unzipped file (.txt file) to build an index that maps words(as strings) to their words reprezentation

In [9]:
glove_100d_path = "glove.6B.100d.txt"
# 1: Create an embedding dictionary
embeddings_index = {}

# 2: Parse the GloVe txt file. Add each word and it's embedding into emeddings_index dictionary
with open(glove_100d_path) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ") # converts coefs strings into float32
    embeddings_index[word] = coefs

print(f'found {len(embeddings_index)} words')

found 400000 words


Next lets build an embedding matrix that will be loaded into an Embedding layer

In [10]:
embedding_dim = 100

# 1: Get the vocabulary from the TextVectorizaton layer
vocabulary = text_vectorization.get_vocabulary()

# 2: Assign to each word a uniquie number
word_index = dict(zip(vocabulary, range(len(vocabulary))))

# 3: Create a matrix filled with zeros
embedding_matrix = np.zeros((max_tokens, embedding_dim))

# 4: Populate embedding_matrxi with GloVe embeddinngs for each word in the vocabulary
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector


In [11]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True
)

This dataset yields inputs that are tf.string tensors and targets that are int32 tensors encoding the value "0" or "1"


All set, now let's try learnign something from this data


In [12]:
inputs = keras.Input(shape=(None, ), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, None, 256)      │      5,120,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 64)             │         73,984 │ embedding_1[0][0],     │
│ (Bidirectional)           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64)             │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1)              │             65 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                      save_best_only=True)
    ]

history = model.fit(int_train_ds.cache(),
  validation_data=int_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 44s 59ms/step - accuracy: 0.6905 - loss: 0.5588 - val_accuracy: 0.8216 - val_loss: 0.3918
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.8571 - loss: 0.3375 - val_accuracy: 0.8608 - val_loss: 0.3330
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8994 - loss: 0.2575 - val_accuracy: 0.8742 - val_loss: 0.3039
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 46ms/step - accuracy: 0.9217 - loss: 0.2111 - val_accuracy: 0.8576 - val_loss: 0.3449
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 44ms/step - accuracy: 0.9444 - loss: 0.1616 - val_accuracy: 0.8804 - val_loss: 0.3133
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 47ms/step - accuracy: 0.9581 - loss: 0.1262 - val_accuracy: 0.8312 - val_loss: 0.5248
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 43ms/step - accuracy: 0.9686 - loss: 0.0980 - val_accuracy: 0.8816 - val_loss: 0.4063
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.9761 - loss: 0.0741 - 

Now let's try to use a Transforer architecture

All the code I've taken from the book 'Deep learning in Python' by François Chollet

In [12]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(
        key_dim=embed_dim, num_heads=num_heads
    )
    self.dense_proj = keras.Sequential([
        layers.Dense(dense_dim, activation='relu'),
        layers.Dense(embed_dim),
    ])
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask=None):
    """
    Mask helps the model ignore padding tokens
    MHA layers expects the mask to be in 3 dimensions, however ours have only 2
   (batch_size, sequence_length). Thus we add a 1 extra dimension so the new shape will be
    (batch_size, 1, sequence_length)
    """
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(
        inputs, inputs, attention_mask=mask
    )
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  def get(self):
    """
    This function saves the models config, making it possible to reload it later
    """
    config = super().get_config()
    config.update({
        'embeded_dim': self.embed_dim,
        'dense_dim': self.dense_dim,
        'num_heads': self.num_heads
    })
    return config


In [13]:
vocab_size = 20000
dense_dim = 32
num_heads = 2
embed_dim = 200

inputs = layers.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, None, 200)           │       4,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, None, 200)           │         335,232 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 200)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,335,433 (16.54 MB)

 Trainable params: 4,335,433 (16.54 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
callbacks = [
    keras.callbacks.ModelCheckpoint('transformer_encoder.keras',
                                    save_best_only=True)
]

history=model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model(
 "transformer_encoder.keras",
 custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 73s 101ms/step - accuracy: 0.5762 - loss: 0.8283 - val_accuracy: 0.7976 - val_loss: 0.4239
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 60s 96ms/step - accuracy: 0.8210 - loss: 0.4046 - val_accuracy: 0.8554 - val_loss: 0.3381
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 97ms/step - accuracy: 0.8450 - loss: 0.3461 - val_accuracy: 0.8524 - val_loss: 0.3372
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 96ms/step - accuracy: 0.8612 - loss: 0.3145 - val_accuracy: 0.8736 - val_loss: 0.2977
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 98ms/step - accuracy: 0.8768 - loss: 0.2917 - val_accuracy: 0.8820 - val_loss: 0.2883
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 62s 99ms/step - accuracy: 0.8849 - loss: 0.2715 - val_accuracy: 0.8798 - val_loss: 0.2882
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 99ms/step - accuracy: 0.9011 - loss: 0.2449 - val_accuracy: 0.8820 - val_loss: 0.2841
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 99ms/step - accuracy: 0.9091 - loss: 0.2241 -

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8818 - loss: 0.2822
Test acc: 0.882


In [16]:
model = keras.models.load_model(
 "transformer_encoder.keras",
 custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8865 - loss: 0.2762
Test acc: 0.882
